# Урок 4. Сверточные нейронные сети

## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
import tensorflow as tf
from multiprocessing import cpu_count
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'

# комбинации параметров нейросети 
best_acc = 0
best_params = {}
params = []
for batch_size in [32, 128]:
    for num_classes in [10, 100]:
        for epochs in [1, 5]:
            for data_augmentation in [True, False]:
                params.append({
                               'batch_size': batch_size, 
                               'num_classes': num_classes, 
                               'epochs': epochs, 
                               'data_augmentation': data_augmentation, 
                              })

for param in params:
    batch_size = param['batch_size']
    num_classes = param['num_classes']
    epochs = param['epochs']
    data_augmentation = param['data_augmentation']

    # разделение тренировочной и тестовой выборки
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # print('x_train shape:', x_train.shape)
    # print(x_train.shape[0], 'тренировочные примеры')
    # print(x_test.shape[0], 'тестовые примеры')
    
    # преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    # конфигурирование слоев нейросети
    model = Sequential()
    
    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    # инициализация RMSprop optimizer
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
    
    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    if not data_augmentation:
        # print('Не используется data augmentation')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True,
                  verbose=0,
                  workers=cpu_count(),
                  use_multiprocessing=True)
    else:
        # print('Использование data augmentation в реальном времени')
        # Препроцессинг и data augmentation в реальном времени:
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            zca_epsilon=1e-06,
            rotation_range=0,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.,
            zoom_range=0.,
            channel_shift_range=0.,
            fill_mode='nearest',
            cval=0.,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=None,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.0)
    
        # запуск data augmentation через fit
        #datagen.fit(x_train)

        # запуск data augmentation через fit_generator
        model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=cpu_count(),
                            use_multiprocessing=True,
                            verbose=0)
    
    # сохранение модели и весов (отключено на для подбора гиперпараметров)
    # if not os.path.isdir(save_dir):
    #     os.makedirs(save_dir)
    # model_path = os.path.join(save_dir, model_name)
    # model.save(model_path)
    # print('сохранить обученную модель как %s ' % model_path)
    
    # проверка работы обученной модели
    scores = model.evaluate(x_test, y_test, verbose=0)
    loss = scores[0]
    accuracy = scores[1]
    param['result_loss'] = loss
    param['result_accuracy'] = accuracy
    # print('Test loss:', scores[0])
    # print('Test accuracy:', scores[1])

    print(f'Точность: {round(scores[1], 4)} при параметрах {param}')
    
    if scores[1] > best_acc:
        best_acc = accuracy
        best_params = param
        
print(f'Лучшая точность: {best_acc} при параметрах {best_params}')

2023-06-21 13:28:41.596667: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-21 13:28:41.646758: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-21 13:28:41.647574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 13:28:42.373443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jupyter/.local/lib/python3.9/site-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_39104/2700436548.py:128: UserWarning: `Model.fit_generator` is deprecated and 

Точность: 0.4196 при параметрах {'batch_size': 32, 'num_classes': 10, 'epochs': 1, 'data_augmentation': True, 'result_loss': 1.6270654201507568, 'result_accuracy': 0.4196000099182129}
Точность: 0.4267 при параметрах {'batch_size': 32, 'num_classes': 10, 'epochs': 1, 'data_augmentation': False, 'result_loss': 1.5857616662979126, 'result_accuracy': 0.42669999599456787}


2023-06-21 13:30:08.668013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Точность: 0.5774 при параметрах {'batch_size': 32, 'num_classes': 10, 'epochs': 5, 'data_augmentation': True, 'result_loss': 1.1993077993392944, 'result_accuracy': 0.5774000287055969}
Точность: 0.626 при параметрах {'batch_size': 32, 'num_classes': 10, 'epochs': 5, 'data_augmentation': False, 'result_loss': 1.0605337619781494, 'result_accuracy': 0.6259999871253967}


2023-06-21 13:37:26.391260: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


...

Точность: 0.5026 при параметрах {'batch_size': 128, 'num_classes': 10, 'epochs': 5, 'data_augmentation': True, 'result_loss': 1.3825068473815918, 'result_accuracy': 0.5026000142097473}
Точность: 0.5212 при параметрах {'batch_size': 128, 'num_classes': 10, 'epochs': 5, 'data_augmentation': False, 'result_loss': 1.319260597229004, 'result_accuracy': 0.5212000012397766}


2023-06-21 13:54:07.456216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Точность: 0.3603 при параметрах {'batch_size': 128, 'num_classes': 100, 'epochs': 1, 'data_augmentation': True, 'result_loss': 1.8687952756881714, 'result_accuracy': 0.3603000044822693}
Точность: 0.3478 при параметрах {'batch_size': 128, 'num_classes': 100, 'epochs': 1, 'data_augmentation': False, 'result_loss': 1.8372331857681274, 'result_accuracy': 0.34779998660087585}


2023-06-21 13:55:21.107560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Точность: 0.4967 при параметрах {'batch_size': 128, 'num_classes': 100, 'epochs': 5, 'data_augmentation': True, 'result_loss': 1.407160758972168, 'result_accuracy': 0.4966999888420105}
Точность: 0.5031 при параметрах {'batch_size': 128, 'num_classes': 100, 'epochs': 5, 'data_augmentation': False, 'result_loss': 1.3762949705123901, 'result_accuracy': 0.5030999779701233}
Лучшая точность: 0.6259999871253967 при параметрах {'batch_size': 32, 'num_classes': 10, 'epochs': 5, 'data_augmentation': False, 'result_loss': 1.0605337619781494, 'result_accuracy': 0.6259999871253967}


# Выводы

увеличение количества эпох увеличивает точность

увеличение batch_size уменьшает точность

отключение data_augmentation улучшает точность